In [ ]:
# Numerische & Kategoriale Analyse der Bundestagsmitglieder
# Analyse von Perioden, Staaten, Parteien, Alter, Bildung und mehr

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from neo4j import GraphDatabase
import warnings
warnings.filterwarnings('ignore')

# Deutsche Labels für bessere Visualisierung
plt.rcParams['font.size'] = 10
plt.rcParams['figure.figsize'] = (12, 8)
sns.set_style("whitegrid")
sns.set_palette("husl")

# Neo4j Verbindung
URI = "bolt://localhost:7687"
AUTH = ("neo4j", "bundestag_password")

print("🏛️ BUNDESTAGSMITGLIEDER ANALYSE")
print("=" * 50)
print("Analysiert werden:")
print("• Altersverteilung über Wahlperioden")
print("• Parteiverteilung und -entwicklung")
print("• Bildungsniveaus und DQR-Klassifikation")
print("• Regionale Verteilung (Bundesländer)")
print("• Vergleich mit Gesamtbevölkerung")
print("• Weitere interessante Korrelationen")
print("=" * 50)


In [ ]:
# Datenextraktion aus Neo4j
def extract_politician_data():
    """Extrahiert umfassende Politiker-Daten aus Neo4j"""
    
    driver = GraphDatabase.driver(URI, auth=AUTH)
    
    query = """
    MATCH (p:Politician)-[:HAS_MANDATE]->(m:Mandate)
    MATCH (m)-[:AFFILIATED_WITH]->(party:Party)
    MATCH (m)-[:IN_PERIOD]->(period:Period)
    OPTIONAL MATCH (m)-[:REPRESENTS_STATE]->(state:State)
    OPTIONAL MATCH (m)-[:REPRESENTS_CONSTITUENCY]->(const:Constituency)
    OPTIONAL MATCH (p)-[:HAS_CLASSIFICATION]->(c:Classification)
    
    RETURN 
        p.firstname as vorname,
        p.lastname as nachname,
        p.full_name as vollname,
        p.birth_year as geburtsjahr,
        p.death_year as todesjahr,
        party.name as partei,
        period.name as periode,
        period.number as periode_nummer,
        state.name as bundesland,
        const.name as wahlkreis,
        m.political_party as partei_detail,
        m.federate_state as bundesland_detail,
        m.constituency as wahlkreis_detail,
        c.dqr_level as dqr_level,
        c.confidence as bildung_confidence,
        c.comment as bildung_kommentar
    """
    
    with driver.session() as session:
        result = session.run(query)
        data = []
        for record in result:
            data.append(dict(record))
    
    driver.close()
    return pd.DataFrame(data)

print("📊 Extrahiere Politiker-Daten aus Neo4j...")
df = extract_politician_data()

print(f"✅ {len(df)} Datensätze geladen")
print(f"🏛️ {df['vollname'].nunique()} einzigartige Politiker")
print(f"🗳️ {df['periode'].nunique()} Wahlperioden")
print(f"🎯 {df['partei'].nunique()} Parteien")
print(f"🗺️ {df['bundesland'].nunique()} Bundesländer")

print("\n📋 Datenstruktur:")
print(df.info())


In [ ]:
# Datenaufbereitung für Analysen
print("🔧 DATENAUFBEREITUNG")
print("=" * 30)

# Alter berechnen (aktuelles Jahr: 2024)
current_year = 2024
df['alter'] = df['geburtsjahr'].apply(lambda x: current_year - x if pd.notna(x) else None)

# Lebende vs. verstorbene Politiker
df['lebend'] = df['todesjahr'].isna()

# Periode als numerisch für Zeitreihen
df['periode_start'] = df['periode'].str.extract(r'(\d{4})').astype(float)

# DQR Level bereinigen und kategorisieren
df['dqr_level_clean'] = pd.to_numeric(df['dqr_level'], errors='coerce')

# Bildungskategorien basierend auf DQR
def categorize_education(dqr):
    if pd.isna(dqr):
        return 'Unbekannt'
    elif dqr <= 2:
        return 'Grundbildung (DQR 1-2)'
    elif dqr <= 4:
        return 'Berufsbildung (DQR 3-4)'
    elif dqr <= 6:
        return 'Hochschulbildung (DQR 5-6)'
    else:
        return 'Promotion/Habilitation (DQR 7-8)'

df['bildungskategorie'] = df['dqr_level_clean'].apply(categorize_education)

# Hauptparteien identifizieren
party_counts = df['partei'].value_counts()
main_parties = party_counts[party_counts >= 50].index.tolist()
df['partei_kategorie'] = df['partei'].apply(lambda x: x if x in main_parties else 'Sonstige')

print("✅ Datenaufbereitung abgeschlossen")
print(f"📊 Altersverteilung: {df['alter'].min():.0f} - {df['alter'].max():.0f} Jahre")
print(f"📈 Perioden: {df['periode_start'].min():.0f} - {df['periode_start'].max():.0f}")
print(f"🎓 Bildungsdaten: {(~df['dqr_level_clean'].isna()).sum()} von {len(df)} Einträgen")
print(f"🏛️ Hauptparteien: {', '.join(main_parties)}")

# Erste Übersicht
print(f"\n📋 DATENÜBERSICHT")
print(f"Altersverteilung:")
print(df['alter'].describe())
print(f"\nParteiverteilung:")
print(df['partei_kategorie'].value_counts().head(10))


In [ ]:
# 1. NUMERISCHE ANALYSE: ALTERSVERTEILUNG
print("📊 1. ALTERSANALYSE DER BUNDESTAGSMITGLIEDER")
print("=" * 50)

# Daten für Altersanalyse vorbereiten
age_data = df.dropna(subset=['alter', 'periode_start'])

fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Altersanalyse der Bundestagsmitglieder', fontsize=16, fontweight='bold')

# 1.1 Histogramm: Gesamte Altersverteilung
axes[0,0].hist(age_data['alter'], bins=30, alpha=0.7, color='steelblue', edgecolor='black')
axes[0,0].set_title('Altersverteilung aller Bundestagsmitglieder')
axes[0,0].set_xlabel('Alter (Jahre)')
axes[0,0].set_ylabel('Häufigkeit')
axes[0,0].axvline(age_data['alter'].mean(), color='red', linestyle='--', label=f'Durchschnitt: {age_data["alter"].mean():.1f}')
axes[0,0].axvline(age_data['alter'].median(), color='orange', linestyle='--', label=f'Median: {age_data["alter"].median():.1f}')
axes[0,0].legend()

# 1.2 Boxplot: Altersverteilung nach Perioden (letzte 10 Perioden)
recent_periods = age_data['periode_start'].nlargest(10).unique()
recent_data = age_data[age_data['periode_start'].isin(recent_periods)]

boxplot_data = [recent_data[recent_data['periode_start'] == period]['alter'].values 
                for period in sorted(recent_periods)]
boxplot_labels = [f"{int(period)}" for period in sorted(recent_periods)]

axes[0,1].boxplot(boxplot_data, labels=boxplot_labels)
axes[0,1].set_title('Altersverteilung nach Wahlperioden (letzte 10)')
axes[0,1].set_xlabel('Wahlperiode (Startjahr)')
axes[0,1].set_ylabel('Alter (Jahre)')
axes[0,1].tick_params(axis='x', rotation=45)

# 1.3 Zeitreihe: Durchschnittsalter über Zeit
age_by_period = age_data.groupby('periode_start')['alter'].agg(['mean', 'median', 'std']).reset_index()

axes[1,0].plot(age_by_period['periode_start'], age_by_period['mean'], 
               marker='o', linewidth=2, label='Durchschnittsalter', color='darkblue')
axes[1,0].fill_between(age_by_period['periode_start'], 
                       age_by_period['mean'] - age_by_period['std'],
                       age_by_period['mean'] + age_by_period['std'],
                       alpha=0.3, color='lightblue', label='±1 Standardabweichung')
axes[1,0].plot(age_by_period['periode_start'], age_by_period['median'], 
               marker='s', linewidth=2, label='Medianalter', color='darkorange')
axes[1,0].set_title('Entwicklung des Durchschnittsalters über Zeit')
axes[1,0].set_xlabel('Wahlperiode (Startjahr)')
axes[1,0].set_ylabel('Alter (Jahre)')
axes[1,0].legend()
axes[1,0].grid(True, alpha=0.3)

# 1.4 Altersverteilung nach Hauptparteien
main_party_data = age_data[age_data['partei_kategorie'].isin(main_parties)]
party_ages = [main_party_data[main_party_data['partei_kategorie'] == party]['alter'].values 
              for party in main_parties]

bp = axes[1,1].boxplot(party_ages, labels=main_parties, patch_artist=True)
colors = plt.cm.Set3(np.linspace(0, 1, len(main_parties)))
for patch, color in zip(bp['boxes'], colors):
    patch.set_facecolor(color)
    
axes[1,1].set_title('Altersverteilung nach Hauptparteien')
axes[1,1].set_xlabel('Partei')
axes[1,1].set_ylabel('Alter (Jahre)')
axes[1,1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

# Statistische Zusammenfassung
print("\n📈 ALTERSSTATISTIKEN:")
print(f"Durchschnittsalter: {age_data['alter'].mean():.1f} Jahre")
print(f"Medianalter: {age_data['alter'].median():.1f} Jahre")
print(f"Jüngster Politiker: {age_data['alter'].min():.0f} Jahre")
print(f"Ältester Politiker: {age_data['alter'].max():.0f} Jahre")
print(f"Standardabweichung: {age_data['alter'].std():.1f} Jahre")

print(f"\n🎯 ALTERSVERTEILUNG NACH PARTEIEN:")
for party in main_parties:
    party_ages = main_party_data[main_party_data['partei_kategorie'] == party]['alter']
    print(f"{party}: Ø {party_ages.mean():.1f} Jahre (Median: {party_ages.median():.1f})")


In [ ]:
# 2. KATEGORIALE ANALYSE: PARTEIVERTEILUNG
print("🎯 2. PARTEIANALYSE ÜBER WAHLPERIODEN")
print("=" * 50)

# Parteidaten vorbereiten
party_data = df.dropna(subset=['partei', 'periode_start'])

fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Parteianalyse der Bundestagsmitglieder', fontsize=16, fontweight='bold')

# 2.1 Balkendiagramm: Gesamte Parteiverteilung
party_counts = party_data['partei_kategorie'].value_counts()
colors = plt.cm.Set3(np.linspace(0, 1, len(party_counts)))

bars = axes[0,0].bar(party_counts.index, party_counts.values, color=colors)
axes[0,0].set_title('Gesamte Parteiverteilung (alle Perioden)')
axes[0,0].set_xlabel('Partei')
axes[0,0].set_ylabel('Anzahl Mandate')
axes[0,0].tick_params(axis='x', rotation=45)

# Werte auf Balken anzeigen
for bar in bars:
    height = bar.get_height()
    axes[0,0].text(bar.get_x() + bar.get_width()/2., height,
                   f'{int(height)}', ha='center', va='bottom')

# 2.2 Entwicklung der Hauptparteien über Zeit
main_parties_evolution = party_data[party_data['partei_kategorie'].isin(main_parties)]
party_by_period = main_parties_evolution.groupby(['periode_start', 'partei_kategorie']).size().unstack(fill_value=0)

party_by_period.plot(kind='line', marker='o', ax=axes[0,1], linewidth=2)
axes[0,1].set_title('Entwicklung der Hauptparteien über Wahlperioden')
axes[0,1].set_xlabel('Wahlperiode (Startjahr)')
axes[0,1].set_ylabel('Anzahl Mandate')
axes[0,1].legend(bbox_to_anchor=(1.05, 1), loc='upper left')
axes[0,1].grid(True, alpha=0.3)

# 2.3 Stacked Bar Chart: Parteiverteilung in den letzten 5 Perioden
recent_periods = sorted(party_data['periode_start'].nlargest(5).unique())
recent_party_data = party_data[party_data['periode_start'].isin(recent_periods)]
party_period_crosstab = pd.crosstab(recent_party_data['periode_start'], recent_party_data['partei_kategorie'])

party_period_crosstab.plot(kind='bar', stacked=True, ax=axes[1,0], 
                          colormap='Set3', width=0.8)
axes[1,0].set_title('Parteiverteilung in den letzten 5 Wahlperioden')
axes[1,0].set_xlabel('Wahlperiode (Startjahr)')
axes[1,0].set_ylabel('Anzahl Mandate')
axes[1,0].legend(bbox_to_anchor=(1.05, 1), loc='upper left')
axes[1,0].tick_params(axis='x', rotation=45)

# 2.4 Pie Chart: Aktuelle Parteiverteilung (letzte Periode)
latest_period = party_data['periode_start'].max()
latest_data = party_data[party_data['periode_start'] == latest_period]
latest_party_counts = latest_data['partei_kategorie'].value_counts()

wedges, texts, autotexts = axes[1,1].pie(latest_party_counts.values, 
                                         labels=latest_party_counts.index,
                                         autopct='%1.1f%%', 
                                         startangle=90,
                                         colors=plt.cm.Set3(np.linspace(0, 1, len(latest_party_counts))))
axes[1,1].set_title(f'Parteiverteilung in der aktuellsten Periode ({int(latest_period)})')

# Legende für Pie Chart
axes[1,1].legend(wedges, [f'{label}: {count}' for label, count in zip(latest_party_counts.index, latest_party_counts.values)],
                 title="Parteien",
                 loc="center left",
                 bbox_to_anchor=(1, 0, 0.5, 1))

plt.tight_layout()
plt.show()

# Parteistatistiken
print(f"\n📊 PARTEISTATISTIKEN:")
print(f"Anzahl verschiedener Parteien: {party_data['partei'].nunique()}")
print(f"Hauptparteien (≥50 Mandate): {len(main_parties)}")
print(f"Größte Partei: {party_counts.index[0]} ({party_counts.iloc[0]} Mandate)")
print(f"Zeitraum: {int(party_data['periode_start'].min())} - {int(party_data['periode_start'].max())}")

print(f"\n🏆 TOP 10 PARTEIEN (Gesamtmandate):")
for i, (party, count) in enumerate(party_counts.head(10).items(), 1):
    percentage = (count / len(party_data)) * 100
    print(f"{i:2d}. {party}: {count} Mandate ({percentage:.1f}%)")

print(f"\n📈 PARTEIVERTEILUNG IN DER AKTUELLSTEN PERIODE ({int(latest_period)}):")
for party, count in latest_party_counts.items():
    percentage = (count / len(latest_data)) * 100
    print(f"{party}: {count} Mandate ({percentage:.1f}%)")


In [ ]:
# 3. BILDUNGSANALYSE: DQR-KLASSIFIKATION UND HOCHSCHULABSCHLÜSSE
print("🎓 3. BILDUNGSANALYSE DER BUNDESTAGSMITGLIEDER")
print("=" * 50)

# Bildungsdaten vorbereiten
education_data = df.dropna(subset=['dqr_level_clean', 'partei_kategorie', 'periode_start'])

fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Bildungsanalyse der Bundestagsmitglieder', fontsize=16, fontweight='bold')

# 3.1 DQR-Level Verteilung
dqr_counts = education_data['bildungskategorie'].value_counts()
colors = ['#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4', '#FFEAA7']

wedges, texts, autotexts = axes[0,0].pie(dqr_counts.values, 
                                         labels=dqr_counts.index,
                                         autopct='%1.1f%%',
                                         startangle=90,
                                         colors=colors[:len(dqr_counts)])
axes[0,0].set_title('Bildungsverteilung nach DQR-Kategorien')

# 3.2 Bildungsniveau nach Parteien
education_party = pd.crosstab(education_data['partei_kategorie'], education_data['bildungskategorie'])
education_party_pct = education_party.div(education_party.sum(axis=1), axis=0) * 100

education_party_pct.plot(kind='bar', stacked=True, ax=axes[0,1], 
                        color=colors[:len(education_party_pct.columns)])
axes[0,1].set_title('Bildungsverteilung nach Parteien (%)')
axes[0,1].set_xlabel('Partei')
axes[0,1].set_ylabel('Prozentanteil')
axes[0,1].legend(bbox_to_anchor=(1.05, 1), loc='upper left')
axes[0,1].tick_params(axis='x', rotation=45)

# 3.3 Entwicklung der Hochschulabschlüsse über Zeit
# Hochschulabschluss = DQR 5-8
education_data['hochschulabschluss'] = education_data['dqr_level_clean'] >= 5
education_data['promotion'] = education_data['dqr_level_clean'] >= 7

education_by_period = education_data.groupby('periode_start').agg({
    'hochschulabschluss': ['count', 'sum'],
    'promotion': 'sum'
}).round(2)

education_by_period.columns = ['gesamt', 'hochschul_abs', 'promotion_abs']
education_by_period['hochschul_pct'] = (education_by_period['hochschul_abs'] / education_by_period['gesamt']) * 100
education_by_period['promotion_pct'] = (education_by_period['promotion_abs'] / education_by_period['gesamt']) * 100

axes[1,0].plot(education_by_period.index, education_by_period['hochschul_pct'], 
               marker='o', linewidth=3, label='Hochschulabschluss (%)', color='darkblue')
axes[1,0].plot(education_by_period.index, education_by_period['promotion_pct'], 
               marker='s', linewidth=3, label='Promotion/Habilitation (%)', color='darkred')
axes[1,0].set_title('Entwicklung der Hochschulabschlüsse über Zeit')
axes[1,0].set_xlabel('Wahlperiode (Startjahr)')
axes[1,0].set_ylabel('Prozentanteil')
axes[1,0].legend()
axes[1,0].grid(True, alpha=0.3)

# 3.4 Detaillierte DQR-Verteilung nach Hauptparteien
main_party_education = education_data[education_data['partei_kategorie'].isin(main_parties)]
dqr_party_detailed = pd.crosstab(main_party_education['partei_kategorie'], 
                                main_party_education['dqr_level_clean'])

dqr_party_detailed.plot(kind='bar', ax=axes[1,1], colormap='viridis')
axes[1,1].set_title('Detaillierte DQR-Level nach Hauptparteien')
axes[1,1].set_xlabel('Partei')
axes[1,1].set_ylabel('Anzahl Politiker')
axes[1,1].legend(title='DQR Level', bbox_to_anchor=(1.05, 1), loc='upper left')
axes[1,1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

# Bildungsstatistiken
print(f"\n📚 BILDUNGSSTATISTIKEN:")
total_with_education = len(education_data)
hochschul_count = education_data['hochschulabschluss'].sum()
promotion_count = education_data['promotion'].sum()

print(f"Politiker mit Bildungsdaten: {total_with_education}")
print(f"Hochschulabschluss: {hochschul_count} ({(hochschul_count/total_with_education)*100:.1f}%)")
print(f"Promotion/Habilitation: {promotion_count} ({(promotion_count/total_with_education)*100:.1f}%)")

print(f"\n🏆 BILDUNGSVERTEILUNG NACH KATEGORIEN:")
for category, count in dqr_counts.items():
    percentage = (count / len(education_data)) * 100
    print(f"{category}: {count} ({percentage:.1f}%)")

print(f"\n🎯 HOCHSCHULABSCHLÜSSE NACH PARTEIEN:")
for party in main_parties:
    party_data = education_data[education_data['partei_kategorie'] == party]
    if len(party_data) > 0:
        hochschul_pct = (party_data['hochschulabschluss'].sum() / len(party_data)) * 100
        promotion_pct = (party_data['promotion'].sum() / len(party_data)) * 100
        print(f"{party}: {hochschul_pct:.1f}% Hochschule, {promotion_pct:.1f}% Promotion")

print(f"\n📈 AKTUELLE BILDUNGSVERTEILUNG (letzte Periode):")
latest_education = education_data[education_data['periode_start'] == education_data['periode_start'].max()]
latest_hochschul = (latest_education['hochschulabschluss'].sum() / len(latest_education)) * 100
latest_promotion = (latest_education['promotion'].sum() / len(latest_education)) * 100
print(f"Hochschulabschluss: {latest_hochschul:.1f}%")
print(f"Promotion/Habilitation: {latest_promotion:.1f}%")


In [ ]:
# 4. REGIONALE ANALYSE: BUNDESLÄNDER UND WAHLKREISE
print("🗺️ 4. REGIONALE ANALYSE DER BUNDESTAGSMITGLIEDER")
print("=" * 50)

# Regionale Daten vorbereiten
regional_data = df.dropna(subset=['bundesland', 'partei_kategorie'])

fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Regionale Analyse der Bundestagsmitglieder', fontsize=16, fontweight='bold')

# 4.1 Verteilung nach Bundesländern
state_counts = regional_data['bundesland'].value_counts()
colors = plt.cm.tab20(np.linspace(0, 1, len(state_counts)))

bars = axes[0,0].barh(range(len(state_counts)), state_counts.values, color=colors)
axes[0,0].set_yticks(range(len(state_counts)))
axes[0,0].set_yticklabels(state_counts.index)
axes[0,0].set_title('Anzahl Bundestagsmitglieder nach Bundesländern')
axes[0,0].set_xlabel('Anzahl Politiker')

# Werte auf Balken anzeigen
for i, bar in enumerate(bars):
    width = bar.get_width()
    axes[0,0].text(width, bar.get_y() + bar.get_height()/2,
                   f'{int(width)}', ha='left', va='center')

# 4.2 Parteiverteilung nach Bundesländern (Top 8 Bundesländer)
top_states = state_counts.head(8).index
top_states_data = regional_data[regional_data['bundesland'].isin(top_states)]
state_party_crosstab = pd.crosstab(top_states_data['bundesland'], top_states_data['partei_kategorie'])

state_party_crosstab.plot(kind='bar', stacked=True, ax=axes[0,1], colormap='Set3')
axes[0,1].set_title('Parteiverteilung in den 8 größten Bundesländern')
axes[0,1].set_xlabel('Bundesland')
axes[0,1].set_ylabel('Anzahl Politiker')
axes[0,1].legend(bbox_to_anchor=(1.05, 1), loc='upper left')
axes[0,1].tick_params(axis='x', rotation=45)

# 4.3 Entwicklung der regionalen Repräsentation über Zeit (Top 5 Bundesländer)
top_5_states = state_counts.head(5).index
regional_time_data = df.dropna(subset=['bundesland', 'periode_start'])
regional_time_data = regional_time_data[regional_time_data['bundesland'].isin(top_5_states)]

state_by_period = regional_time_data.groupby(['periode_start', 'bundesland']).size().unstack(fill_value=0)
state_by_period = state_by_period[top_5_states]  # Reihenfolge beibehalten

state_by_period.plot(kind='line', marker='o', ax=axes[1,0], linewidth=2)
axes[1,0].set_title('Entwicklung der Repräsentation (Top 5 Bundesländer)')
axes[1,0].set_xlabel('Wahlperiode (Startjahr)')
axes[1,0].set_ylabel('Anzahl Politiker')
axes[1,0].legend(bbox_to_anchor=(1.05, 1), loc='upper left')
axes[1,0].grid(True, alpha=0.3)

# 4.4 Bildungsniveau nach Bundesländern (Top 6 Bundesländer)
top_6_states = state_counts.head(6).index
education_regional = df.dropna(subset=['bundesland', 'dqr_level_clean'])
education_regional = education_regional[education_regional['bundesland'].isin(top_6_states)]
education_regional['hochschulabschluss'] = education_regional['dqr_level_clean'] >= 5

education_by_state = education_regional.groupby('bundesland').agg({
    'hochschulabschluss': ['count', 'sum']
}).round(2)

education_by_state.columns = ['gesamt', 'hochschul_abs']
education_by_state['hochschul_pct'] = (education_by_state['hochschul_abs'] / education_by_state['gesamt']) * 100

bars = axes[1,1].bar(range(len(education_by_state)), education_by_state['hochschul_pct'], 
                     color=plt.cm.viridis(np.linspace(0, 1, len(education_by_state))))
axes[1,1].set_xticks(range(len(education_by_state)))
axes[1,1].set_xticklabels(education_by_state.index, rotation=45)
axes[1,1].set_title('Anteil Hochschulabschlüsse nach Bundesländern')
axes[1,1].set_xlabel('Bundesland')
axes[1,1].set_ylabel('Prozent mit Hochschulabschluss')

# Werte auf Balken anzeigen
for i, bar in enumerate(bars):
    height = bar.get_height()
    axes[1,1].text(bar.get_x() + bar.get_width()/2., height,
                   f'{height:.1f}%', ha='center', va='bottom')

plt.tight_layout()
plt.show()

# Regionale Statistiken
print(f"\n🗺️ REGIONALE STATISTIKEN:")
print(f"Anzahl Bundesländer: {regional_data['bundesland'].nunique()}")
print(f"Größtes Bundesland: {state_counts.index[0]} ({state_counts.iloc[0]} Politiker)")
print(f"Kleinstes Bundesland: {state_counts.index[-1]} ({state_counts.iloc[-1]} Politiker)")

print(f"\n🏆 TOP 10 BUNDESLÄNDER (Anzahl Politiker):")
for i, (state, count) in enumerate(state_counts.head(10).items(), 1):
    percentage = (count / len(regional_data)) * 100
    print(f"{i:2d}. {state}: {count} Politiker ({percentage:.1f}%)")

print(f"\n🎓 BILDUNGSNIVEAU NACH BUNDESLÄNDERN (Top 6):")
for state in education_by_state.index:
    gesamt = education_by_state.loc[state, 'gesamt']
    pct = education_by_state.loc[state, 'hochschul_pct']
    print(f"{state}: {pct:.1f}% Hochschulabschluss ({gesamt} Politiker)")

# Ost-West Vergleich (vereinfacht)
ost_states = ['Sachsen', 'Thüringen', 'Sachsen-Anhalt', 'Brandenburg', 'Mecklenburg-Vorpommern', 'Berlin']
west_data = regional_data[~regional_data['bundesland'].isin(ost_states)]
ost_data = regional_data[regional_data['bundesland'].isin(ost_states)]

print(f"\n🔄 OST-WEST VERGLEICH:")
print(f"Westdeutschland: {len(west_data)} Politiker ({(len(west_data)/len(regional_data))*100:.1f}%)")
print(f"Ostdeutschland: {len(ost_data)} Politiker ({(len(ost_data)/len(regional_data))*100:.1f}%)")

if len(ost_data) > 0 and len(west_data) > 0:
    # Bildungsvergleich Ost-West
    west_education = df[(~df['bundesland'].isin(ost_states)) & (~df['dqr_level_clean'].isna())]
    ost_education = df[(df['bundesland'].isin(ost_states)) & (~df['dqr_level_clean'].isna())]
    
    if len(west_education) > 0 and len(ost_education) > 0:
        west_hochschul = (west_education['dqr_level_clean'] >= 5).sum() / len(west_education) * 100
        ost_hochschul = (ost_education['dqr_level_clean'] >= 5).sum() / len(ost_education) * 100
        
        print(f"Hochschulabschluss West: {west_hochschul:.1f}%")
        print(f"Hochschulabschluss Ost: {ost_hochschul:.1f}%")


In [ ]:
# 5. VERGLEICH MIT DER GESAMTBEVÖLKERUNG
print("👥 5. VERGLEICH MIT DER DEUTSCHEN GESAMTBEVÖLKERUNG")
print("=" * 50)

# Simulierte Bevölkerungsdaten (basierend auf Statistisches Bundesamt 2023)
# Diese können durch echte Daten ersetzt werden
population_data = {
    'alter_durchschnitt': 44.6,  # Durchschnittsalter Deutschland 2023
    'hochschulabschluss_pct': 18.5,  # Anteil mit Hochschulabschluss (25-64 Jahre)
    'promotion_pct': 1.2,  # Anteil mit Promotion
    'geschlecht_frauen_pct': 50.7,  # Frauenanteil
    'bildung_verteilung': {
        'ohne_abschluss': 4.0,
        'hauptschule': 30.5,
        'realschule': 22.8,
        'abitur': 24.2,
        'hochschule': 18.5
    },
    'ost_west_verteilung': {
        'west': 79.8,
        'ost': 20.2
    }
}

fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Vergleich Bundestagsmitglieder vs. Gesamtbevölkerung', fontsize=16, fontweight='bold')

# 5.1 Altersvergleich
bundestag_alter = df['alter'].dropna().mean()
population_alter = population_data['alter_durchschnitt']

ages_comparison = ['Bundestag', 'Gesamtbevölkerung']
ages_values = [bundestag_alter, population_alter]

bars = axes[0,0].bar(ages_comparison, ages_values, color=['#FF6B6B', '#4ECDC4'])
axes[0,0].set_title('Durchschnittsalter: Bundestag vs. Gesamtbevölkerung')
axes[0,0].set_ylabel('Durchschnittsalter (Jahre)')

for bar, value in zip(bars, ages_values):
    axes[0,0].text(bar.get_x() + bar.get_width()/2., value,
                   f'{value:.1f}', ha='center', va='bottom', fontweight='bold')

# 5.2 Bildungsvergleich
education_bundestag = df.dropna(subset=['dqr_level_clean'])
bundestag_hochschul = (education_bundestag['dqr_level_clean'] >= 5).sum() / len(education_bundestag) * 100
bundestag_promotion = (education_bundestag['dqr_level_clean'] >= 7).sum() / len(education_bundestag) * 100

education_comparison = ['Hochschulabschluss', 'Promotion']
bundestag_values = [bundestag_hochschul, bundestag_promotion]
population_values = [population_data['hochschulabschluss_pct'], population_data['promotion_pct']]

x = np.arange(len(education_comparison))
width = 0.35

bars1 = axes[0,1].bar(x - width/2, bundestag_values, width, label='Bundestag', color='#FF6B6B')
bars2 = axes[0,1].bar(x + width/2, population_values, width, label='Gesamtbevölkerung', color='#4ECDC4')

axes[0,1].set_title('Bildungsniveau: Bundestag vs. Gesamtbevölkerung')
axes[0,1].set_ylabel('Prozentanteil')
axes[0,1].set_xticks(x)
axes[0,1].set_xticklabels(education_comparison)
axes[0,1].legend()

# Werte auf Balken anzeigen
for bars in [bars1, bars2]:
    for bar in bars:
        height = bar.get_height()
        axes[0,1].text(bar.get_x() + bar.get_width()/2., height,
                       f'{height:.1f}%', ha='center', va='bottom')

# 5.3 Regionale Repräsentation vs. Bevölkerungsverteilung
# Vereinfachter Ost-West Vergleich
ost_states = ['Sachsen', 'Thüringen', 'Sachsen-Anhalt', 'Brandenburg', 'Mecklenburg-Vorpommern', 'Berlin']
bundestag_regional = df.dropna(subset=['bundesland'])
bundestag_ost = (bundestag_regional['bundesland'].isin(ost_states)).sum() / len(bundestag_regional) * 100
bundestag_west = 100 - bundestag_ost

regional_comparison = ['Westdeutschland', 'Ostdeutschland']
bundestag_regional_values = [bundestag_west, bundestag_ost]
population_regional_values = [population_data['ost_west_verteilung']['west'], 
                             population_data['ost_west_verteilung']['ost']]

x = np.arange(len(regional_comparison))
bars1 = axes[1,0].bar(x - width/2, bundestag_regional_values, width, label='Bundestag', color='#FF6B6B')
bars2 = axes[1,0].bar(x + width/2, population_regional_values, width, label='Gesamtbevölkerung', color='#4ECDC4')

axes[1,0].set_title('Regionale Verteilung: Bundestag vs. Gesamtbevölkerung')
axes[1,0].set_ylabel('Prozentanteil')
axes[1,0].set_xticks(x)
axes[1,0].set_xticklabels(regional_comparison)
axes[1,0].legend()

for bars in [bars1, bars2]:
    for bar in bars:
        height = bar.get_height()
        axes[1,0].text(bar.get_x() + bar.get_width()/2., height,
                       f'{height:.1f}%', ha='center', va='bottom')

# 5.4 Repräsentativitäts-Index (Bundestag/Bevölkerung Verhältnis)
metrics = ['Hochschulabschluss', 'Promotion', 'Durchschnittsalter\n(normiert)', 'Ostdeutschland']
ratios = [
    bundestag_hochschul / population_data['hochschulabschluss_pct'],
    bundestag_promotion / population_data['promotion_pct'],
    bundestag_alter / population_data['alter_durchschnitt'],
    bundestag_ost / population_data['ost_west_verteilung']['ost']
]

colors_ratio = ['green' if r < 1.5 else 'orange' if r < 3 else 'red' for r in ratios]
bars = axes[1,1].bar(metrics, ratios, color=colors_ratio, alpha=0.7)
axes[1,1].axhline(y=1, color='black', linestyle='--', alpha=0.5, label='Repräsentativ (1:1)')
axes[1,1].set_title('Repräsentativitäts-Index\n(Bundestag/Bevölkerung)')
axes[1,1].set_ylabel('Verhältnis (1 = repräsentativ)')
axes[1,1].legend()
axes[1,1].tick_params(axis='x', rotation=45)

for bar, ratio in zip(bars, ratios):
    axes[1,1].text(bar.get_x() + bar.get_width()/2., ratio,
                   f'{ratio:.1f}x', ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.show()

# Detaillierte Vergleichsstatistiken
print(f"\n📊 DETAILLIERTER VERGLEICH:")
print(f"{'Merkmal':<25} {'Bundestag':<15} {'Bevölkerung':<15} {'Verhältnis':<10}")
print("-" * 70)
print(f"{'Durchschnittsalter':<25} {bundestag_alter:.1f} Jahre{'':<6} {population_alter:.1f} Jahre{'':<6} {bundestag_alter/population_alter:.2f}x")
print(f"{'Hochschulabschluss':<25} {bundestag_hochschul:.1f}%{'':<10} {population_data['hochschulabschluss_pct']:.1f}%{'':<10} {bundestag_hochschul/population_data['hochschulabschluss_pct']:.1f}x")
print(f"{'Promotion':<25} {bundestag_promotion:.1f}%{'':<10} {population_data['promotion_pct']:.1f}%{'':<11} {bundestag_promotion/population_data['promotion_pct']:.1f}x")
print(f"{'Ostdeutschland':<25} {bundestag_ost:.1f}%{'':<10} {population_data['ost_west_verteilung']['ost']:.1f}%{'':<10} {bundestag_ost/population_data['ost_west_verteilung']['ost']:.1f}x")

print(f"\n🔍 INTERPRETATIONEN:")
print(f"• Überrepräsentation Akademiker: Bundestag hat {bundestag_hochschul/population_data['hochschulabschluss_pct']:.1f}x mehr Hochschulabsolventen")
print(f"• Extreme Überrepräsentation Promovierte: {bundestag_promotion/population_data['promotion_pct']:.1f}x mehr als in der Bevölkerung")
print(f"• Altersstruktur: Bundestag ist {bundestag_alter - population_alter:.1f} Jahre älter im Durchschnitt")
print(f"• Regionale Balance: Ostdeutschland ist {'über' if bundestag_ost > population_data['ost_west_verteilung']['ost'] else 'unter'}repräsentiert")

# Empfohlene Datensätze für genauere Analysen
print(f"\n📚 EMPFOHLENE DATENSÄTZE FÜR PRÄZISERE VERGLEICHE:")
print("1. Destatis Mikrozensus (Bildung, Alter, Region)")
print("2. GESIS Allbus (Soziale Einstellungen)")
print("3. Eurostat Education Statistics")
print("4. OECD Education at a Glance")
print("5. Statistisches Bundesamt Bevölkerungsfortschreibung")


In [ ]:
# 6. ERWEITERTE ANALYSEN UND INTERESSANTE INSIGHTS
print("🔍 6. ERWEITERTE ANALYSEN UND KORRELATIONEN")
print("=" * 50)

fig, axes = plt.subplots(2, 3, figsize=(20, 12))
fig.suptitle('Erweiterte Analysen der Bundestagsmitglieder', fontsize=16, fontweight='bold')

# 6.1 Karrierelänge-Analyse
career_data = df.groupby('vollname').agg({
    'periode_start': ['min', 'max', 'count'],
    'alter': 'first',
    'partei_kategorie': 'first',
    'dqr_level_clean': 'first'
}).round(2)

career_data.columns = ['erste_periode', 'letzte_periode', 'anzahl_perioden', 'alter', 'partei', 'dqr_level']
career_data['karriere_jahre'] = career_data['letzte_periode'] - career_data['erste_periode']
career_data = career_data[career_data['anzahl_perioden'] >= 2]  # Nur Politiker mit mehreren Perioden

axes[0,0].scatter(career_data['alter'], career_data['karriere_jahre'], alpha=0.6, s=50)
axes[0,0].set_title('Karrierelänge vs. Alter bei Erstmandat')
axes[0,0].set_xlabel('Alter bei Erstmandat')
axes[0,0].set_ylabel('Karrierelänge (Jahre)')
axes[0,0].grid(True, alpha=0.3)

# Trendlinie
if len(career_data) > 1:
    z = np.polyfit(career_data['alter'].dropna(), career_data['karriere_jahre'].dropna(), 1)
    p = np.poly1d(z)
    axes[0,0].plot(career_data['alter'].dropna(), p(career_data['alter'].dropna()), "r--", alpha=0.8)

# 6.2 Bildung vs. Partei Heatmap
education_party_detailed = df.dropna(subset=['partei_kategorie', 'dqr_level_clean'])
main_parties_edu = education_party_detailed[education_party_detailed['partei_kategorie'].isin(main_parties)]
heatmap_data = pd.crosstab(main_parties_edu['partei_kategorie'], main_parties_edu['dqr_level_clean'], normalize='index') * 100

sns.heatmap(heatmap_data, annot=True, fmt='.1f', cmap='YlOrRd', ax=axes[0,1])
axes[0,1].set_title('Bildungsniveau nach Parteien (%)')
axes[0,1].set_xlabel('DQR Level')
axes[0,1].set_ylabel('Partei')

# 6.3 Alters-Bildungs-Korrelation
age_education = df.dropna(subset=['alter', 'dqr_level_clean'])
if len(age_education) > 0:
    correlation = age_education['alter'].corr(age_education['dqr_level_clean'])
    
    axes[0,2].scatter(age_education['alter'], age_education['dqr_level_clean'], alpha=0.6, s=30)
    axes[0,2].set_title(f'Alter vs. Bildungsniveau\n(Korrelation: {correlation:.3f})')
    axes[0,2].set_xlabel('Alter')
    axes[0,2].set_ylabel('DQR Level')
    axes[0,2].grid(True, alpha=0.3)

# 6.4 Entwicklung der Diversität über Zeit
diversity_data = df.dropna(subset=['periode_start', 'partei_kategorie'])
diversity_by_period = diversity_data.groupby('periode_start')['partei_kategorie'].nunique()

axes[1,0].plot(diversity_by_period.index, diversity_by_period.values, marker='o', linewidth=3, color='darkgreen')
axes[1,0].set_title('Parteiendiversität über Zeit')
axes[1,0].set_xlabel('Wahlperiode (Startjahr)')
axes[1,0].set_ylabel('Anzahl verschiedener Parteien')
axes[1,0].grid(True, alpha=0.3)

# 6.5 Regionale Bildungsunterschiede (erweitert)
regional_education_detailed = df.dropna(subset=['bundesland', 'dqr_level_clean'])
top_states_detailed = regional_education_detailed['bundesland'].value_counts().head(8).index

regional_edu_stats = []
for state in top_states_detailed:
    state_data = regional_education_detailed[regional_education_detailed['bundesland'] == state]
    avg_dqr = state_data['dqr_level_clean'].mean()
    hochschul_pct = (state_data['dqr_level_clean'] >= 5).sum() / len(state_data) * 100
    regional_edu_stats.append({'Bundesland': state, 'Durchschnitt_DQR': avg_dqr, 'Hochschul_Prozent': hochschul_pct})

regional_edu_df = pd.DataFrame(regional_edu_stats)
bars = axes[1,1].bar(range(len(regional_edu_df)), regional_edu_df['Durchschnitt_DQR'], 
                     color=plt.cm.plasma(np.linspace(0, 1, len(regional_edu_df))))
axes[1,1].set_xticks(range(len(regional_edu_df)))
axes[1,1].set_xticklabels(regional_edu_df['Bundesland'], rotation=45)
axes[1,1].set_title('Durchschnittliches DQR-Level nach Bundesländern')
axes[1,1].set_ylabel('Durchschnittliches DQR Level')

# 6.6 Mandatsdauer vs. Bildung
mandate_education = df.dropna(subset=['vollname', 'dqr_level_clean'])
mandate_counts = mandate_education.groupby('vollname').agg({
    'periode_start': 'count',
    'dqr_level_clean': 'first'
}).rename(columns={'periode_start': 'anzahl_mandate'})

# Boxplot für verschiedene Bildungsniveaus
education_levels = sorted(mandate_counts['dqr_level_clean'].unique())
mandate_by_education = [mandate_counts[mandate_counts['dqr_level_clean'] == level]['anzahl_mandate'].values 
                       for level in education_levels]

bp = axes[1,2].boxplot(mandate_by_education, labels=education_levels, patch_artist=True)
colors = plt.cm.viridis(np.linspace(0, 1, len(education_levels)))
for patch, color in zip(bp['boxes'], colors):
    patch.set_facecolor(color)

axes[1,2].set_title('Mandatsdauer nach Bildungsniveau')
axes[1,2].set_xlabel('DQR Level')
axes[1,2].set_ylabel('Anzahl Mandate')

plt.tight_layout()
plt.show()

# Erweiterte Statistiken und Insights
print(f"\n🎯 INTERESSANTE ERKENNTNISSE:")

# Karriere-Insights
if len(career_data) > 0:
    longest_career = career_data['karriere_jahre'].max()
    most_periods = career_data['anzahl_perioden'].max()
    avg_career = career_data['karriere_jahre'].mean()
    
    print(f"• Längste Karriere: {longest_career:.0f} Jahre")
    print(f"• Meiste Perioden: {most_periods:.0f}")
    print(f"• Durchschnittliche Karrierelänge: {avg_career:.1f} Jahre")

# Bildungs-Partei Insights
if len(heatmap_data) > 0:
    highest_education_party = heatmap_data[8].idxmax() if 8 in heatmap_data.columns else "N/A"
    if highest_education_party != "N/A":
        print(f"• Partei mit höchstem Promotionsanteil: {highest_education_party}")

# Korrelations-Insights
if 'correlation' in locals():
    if abs(correlation) > 0.1:
        direction = "positive" if correlation > 0 else "negative"
        print(f"• {direction.title()} Korrelation zwischen Alter und Bildung: {correlation:.3f}")

# Diversitäts-Insights
diversity_trend = diversity_by_period.iloc[-1] - diversity_by_period.iloc[0]
print(f"• Parteiendiversität: {'Zunahme' if diversity_trend > 0 else 'Abnahme'} um {abs(diversity_trend):.0f} Parteien")

# Regionale Bildungsunterschiede
if len(regional_edu_df) > 0:
    highest_edu_state = regional_edu_df.loc[regional_edu_df['Durchschnitt_DQR'].idxmax(), 'Bundesland']
    lowest_edu_state = regional_edu_df.loc[regional_edu_df['Durchschnitt_DQR'].idxmin(), 'Bundesland']
    print(f"• Höchstes Bildungsniveau: {highest_edu_state}")
    print(f"• Niedrigstes Bildungsniveau: {lowest_edu_state}")

print(f"\n📈 WEITERE ANALYSEMÖGLICHKEITEN:")
print("1. Geschlechteranalyse (falls Geschlechtsdaten verfügbar)")
print("2. Berufshintergrund-Analyse vor politischer Karriere")
print("3. Netzwerk-Analyse zwischen Politikern")
print("4. Zeitreihenanalyse politischer Themen")
print("5. Sentiment-Analyse der Politiker-Inhalte")
print("6. Wahlkreis-Demografie vs. Politiker-Profile")
print("7. Internationale Vergleiche mit anderen Parlamenten")

print(f"\n💡 EMPFEHLUNGEN FÜR WEITERE FORSCHUNG:")
print("• Integration von Wahlkreis-Demografiedaten")
print("• Analyse der Berufshintergründe vor der Politik")
print("• Longitudinale Studien über Karriereverläufe")
print("• Vergleich mit anderen demokratischen Systemen")
print("• Einfluss der Bildung auf Abstimmungsverhalten")


In [ ]:
# 7. ZUSAMMENFASSUNG UND SCHLUSSFOLGERUNGEN
print("📋 7. ZUSAMMENFASSUNG DER ANALYSE")
print("=" * 50)

# Gesamtstatistik erstellen
total_politicians = df['vollname'].nunique()
total_mandates = len(df)
total_periods = df['periode'].nunique()
total_parties = df['partei'].nunique()
total_states = df['bundesland'].nunique()

print(f"🏛️ DATENBASIS:")
print(f"• {total_politicians:,} einzigartige Bundestagsmitglieder")
print(f"• {total_mandates:,} Mandate insgesamt")
print(f"• {total_periods} Wahlperioden")
print(f"• {total_parties} verschiedene Parteien")
print(f"• {total_states} Bundesländer")

print(f"\n📊 KERNERKENNTNISSE:")

# Alterserkenntnisse
age_data_summary = df.dropna(subset=['alter'])
if len(age_data_summary) > 0:
    avg_age = age_data_summary['alter'].mean()
    print(f"• Durchschnittsalter: {avg_age:.1f} Jahre")
    print(f"  → {avg_age - 44.6:.1f} Jahre älter als deutsche Gesamtbevölkerung")

# Bildungserkenntnisse
education_summary = df.dropna(subset=['dqr_level_clean'])
if len(education_summary) > 0:
    hochschul_rate = (education_summary['dqr_level_clean'] >= 5).sum() / len(education_summary) * 100
    promotion_rate = (education_summary['dqr_level_clean'] >= 7).sum() / len(education_summary) * 100
    print(f"• Hochschulabschluss: {hochschul_rate:.1f}% (vs. 18.5% Bevölkerung)")
    print(f"  → {hochschul_rate/18.5:.1f}x höhere Akademikerrate")
    print(f"• Promotion/Habilitation: {promotion_rate:.1f}% (vs. 1.2% Bevölkerung)")
    print(f"  → {promotion_rate/1.2:.1f}x höhere Promotionsrate")

# Parteierkenntnisse
party_summary = df['partei_kategorie'].value_counts()
print(f"• Größte Partei: {party_summary.index[0]} ({party_summary.iloc[0]:,} Mandate)")
print(f"• Parteikonzentration: Top 3 Parteien haben {(party_summary.head(3).sum()/len(df)*100):.1f}% aller Mandate")

# Regionale Erkenntnisse
regional_summary = df.dropna(subset=['bundesland'])
if len(regional_summary) > 0:
    state_counts_summary = regional_summary['bundesland'].value_counts()
    print(f"• Größtes Bundesland: {state_counts_summary.index[0]} ({state_counts_summary.iloc[0]:,} Politiker)")
    
    # Ost-West Balance
    ost_states = ['Sachsen', 'Thüringen', 'Sachsen-Anhalt', 'Brandenburg', 'Mecklenburg-Vorpommern', 'Berlin']
    ost_count = (regional_summary['bundesland'].isin(ost_states)).sum()
    ost_percentage = (ost_count / len(regional_summary)) * 100
    print(f"• Ostdeutschland: {ost_percentage:.1f}% (vs. 20.2% Bevölkerungsanteil)")

print(f"\n🔍 WICHTIGSTE TRENDS:")
print("• Extreme Überrepräsentation von Akademikern im Bundestag")
print("• Besonders hoher Anteil an Promovierten")
print("• Durchschnittsalter deutlich über Bevölkerungsschnitt")
print("• Regionale Repräsentation weitgehend proportional")
print("• Parteiendiversität hat über Zeit zugenommen")
print("• Bildungsniveau variiert zwischen Parteien")

print(f"\n⚠️ LIMITATIONEN DER ANALYSE:")
print("• Fehlende Geschlechtsdaten")
print("• Bildungsdaten nicht vollständig (nur KI-klassifiziert)")
print("• Vereinfachte Bevölkerungsvergleichsdaten")
print("• Keine Berücksichtigung von Berufshintergründen")
print("• Zeitreihen teilweise unvollständig")

print(f"\n🎯 EMPFEHLUNGEN FÜR POLITIKWISSENSCHAFT:")
print("1. Untersuchung der Repräsentationslücke bei Bildung")
print("2. Analyse der Auswirkungen auf Politikgestaltung")
print("3. Vergleich mit anderen Demokratien")
print("4. Langzeitstudie über Diversitätsentwicklung")
print("5. Einfluss der Bildung auf Abstimmungsverhalten")

print(f"\n📈 MÖGLICHE FOLLOW-UP STUDIEN:")
print("• Korrelation Bildung ↔ Abstimmungsverhalten")
print("• Netzwerkanalyse zwischen Politikern")
print("• Einfluss der Herkunft auf Politikpositionen")
print("• Karriereverläufe und Aufstiegsmuster")
print("• Vergleich Bundestag ↔ Landesparlamente")

print(f"\n✅ ANALYSE ABGESCHLOSSEN")
print("=" * 50)
print("Diese umfassende Analyse bietet eine solide Grundlage für weitere")
print("politikwissenschaftliche Forschung und demokratietheoretische Diskussionen.")
print("Die Daten zeigen deutliche Muster in der Zusammensetzung des Bundestags")
print("und wichtige Unterschiede zur Gesamtbevölkerung auf.")
